# Eminem's Album Trends using Spotify APIs

Album trend analysis posted on:
+ https://kaivalyapowale.com/2020/01/25/eminems-album-trends-and-music-to-be-murdered-by-2020/


A part of GitHub repository:
+ https://github.com/kaivalyapowale/Spotify-Eminem-Album-Trends


By Kaivalya Powale
+ http://kaivalyapowale.com/
+ https://www.linkedin.com/in/kaivalya-powale/
+ https://public.tableau.com/profile/kaivalya.powale#!/

### Importing libraries

import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import base64
import time

I have saved my Spotify Client ID and Secret ID in a csv (for security).
If you don't know how to get the Client ID and Secret ID for yourself, refer to:

https://kaivalyapowale.com/2020/01/18/how-to-access-spotify-api/

It just takes 2 minutes, I promise!

In [ ]:
spotify_ids = pd.read_csv('/Users/kaivalya/Spotify/Repo/spotify_ids.csv')

### Getting Spotify Access Token

You have to encode the Client ID and Secret, then make a request for acces token.

In [3]:
# Defining base64 encoding of the IDs
def base64_encode(client_id,client_secret):
    encodedData = base64.b64encode(bytes(f"{client_id}:{client_secret}", "ISO-8859-1")).decode("ascii")
    authorization_header_string = f"{encodedData}"
    return(authorization_header_string)

In [4]:
def accesstoken(client_id, client_secret):
    header_string= base64_encode(client_id,client_secret)
    headers = {
        'Authorization': 'Basic '+header_string,
    }
    
    data = {
        'grant_type': 'client_credentials'
    }
    
    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)
    access_token = json.loads(response.text)
    access_token = access_token['access_token']
    return(access_token)

In [10]:
access_token = accesstoken(str(spotify_ids['code'][0]),str(spotify_ids['code'][1]))
access_token

'BQAvOuzVkdeYN26-5vEyPI5mghVn3bFmevQe-XkFruSBT_pmnLzaVdr0Jq-cQ2O7vvtqcPR-GyZW0RVd6Lc'

### Making a function to get playlist information 

We're going to make 2 functions that ask for a Playlist's tracks and then get Audio Features for those tracks!
We will use these functions later, in a bigger function. This will help us automate these repetitive steps!

In [5]:
def get_playlist(playlist_id,ns):
    headers = {
        'Authorization': 'Bearer '+access_token,
    }
    playlist_response = requests.get('https://api.spotify.com/v1/playlists/'+playlist_id+'/tracks?limit='+ns, headers=headers)
    
    playlist_response = json.loads(playlist_response.text)
    playlist_response = playlist_response['items']
    playlist_response = pd.DataFrame.from_dict(json_normalize(playlist_response), orient='columns')
    return(playlist_response)

In [6]:
def get_playlists_tracks(playlist_df):
    
    track_ids = list(playlist_df['track.id'])
    track_ids = ','.join(track_ids)
    
    headers = {
        'Authorization': 'Bearer '+access_token,
    }
    track_response = requests.get('https://api.spotify.com/v1/audio-features/?ids='+track_ids, headers=headers)
    track_response = json.loads(track_response.text)
    track_response = track_response['audio_features']
    track_response = pd.DataFrame.from_dict(json_normalize(track_response), orient='columns')
    
    playlist_df = pd.merge(playlist_df, track_response, left_on='track.id', right_on='id', how='inner')
    return(playlist_df)

### Making a function to get album information 

We're going to make 2 functions that ask for a Album's tracks and then get Audio Features for those tracks!
We will use these functions later, in a bigger function. This will help us automate these repetitive steps!

In [7]:
def get_album(album_id,ns):
    headers = {
        'Authorization': 'Bearer '+access_token,
    }
    album_response = requests.get('https://api.spotify.com/v1/albums/'+album_id+'/tracks?limit='+ns, headers=headers)
    
    album_response = json.loads(album_response.text)
    album_response = album_response['items']
    album_response = pd.DataFrame.from_dict(json_normalize(album_response), orient='columns')
    return(album_response)

In [8]:
def get_albums_tracks(playlist_df):
    
    track_ids = list(playlist_df['id'])
    track_ids = ','.join(track_ids)
    
    headers = {
        'Authorization': 'Bearer '+access_token,
    }
    track_response = requests.get('https://api.spotify.com/v1/audio-features/?ids='+track_ids, headers=headers)
    track_response = json.loads(track_response.text)
    track_response = track_response['audio_features']
    track_response = pd.DataFrame.from_dict(json_normalize(track_response), orient='columns')
    
    playlist_df = pd.merge(playlist_df, track_response, left_on='id', right_on='id', how='inner')
    return(playlist_df)

### Audio Features Function

This function has automated almost everything.
All you have to do is run it and enter the ID for the Playlist or Album you want information for!

In [9]:
def audio_features():
    
    pora = input('Is it a Playlist or an Album? p/a')
    pora=str(pora)
    while pora not in ('p','a'):
        pora = input('Please re-enter if it is a Playlist or an Album? p = playlist / a = album')
        pora=str(pora)
    if pora == 'p':
        pora_id = input('Enter Playlist ID')
        pora_id = str(pora_id)
    else:
        pora_id = input('Enter Album ID')
        pora_id = str(pora_id)
    number_of_songs = input('Enter number of songs you want info for (between 1 and 100)')
    number_of_songs = str(number_of_songs)
    relevance = input('Get only relevant data? y/n')
    relevance = str(relevance.lower())
    while relevance not in ('y','n'):
        print('Relevance will give you only the data that can be used for analysis. Other information like urls or duplicate information will be deleted.')
        relevance = input('Please re-enter if want only relevant data? y = yes / n = no')
        relevance = str(relevance.lower())
    w2csv = input('Write to csv? y/n')
    w2csv = str(w2csv)
    while w2csv not in ('y','n'):
        print('Writing to csv will enable you to locally store the data and use it later without re-retrieval')
        w2csv = input('Please re-enter if you want to write to csv? y/n')
        w2csv = str(w2csv)
    if w2csv == 'y':
        name = input('Save by name?')
        name = str(name)
    
    if pora == 'p':
        myplaylist_df = get_playlist(pora_id, number_of_songs)
        myplaylist_df = get_playlists_tracks(myplaylist_df)
    else:
        myplaylist_df = get_album(pora_id, number_of_songs)
        myplaylist_df = get_albums_tracks(myplaylist_df)
    
    if relevance=='y':
        if pora == 'p':
            i=[1,2,3,4,5,6,7,9,10,11,12,13,14,17,19,20,23,25,27,28,29,31,34,35,36,37,38,39,52,53,54]
            myplaylist_df = myplaylist_df.drop(columns=myplaylist_df.columns[i])
        else:
            i=[0,5,7,9,11,12,13,25,26,27,28,29]
            myplaylist_df = myplaylist_df.drop(columns=myplaylist_df.columns[i])
    
    if w2csv == 'y':
        export_csv = myplaylist_df.to_csv (r'/Users/'+name+'.csv', index = None, header=True)
    
    return(myplaylist_df)

+ When you call the function, it will ask if you want information for a Playlist or an Album
+ You enter the ID
+ Number of songs you want info for (if you're not sure just enter the upper limit)
+ If you are using this for the Eminem dashboard, I have given an option to get the relevant data. Feel free to say 'n' in case you want to get all info given by Spotify.
+ You can say 'y' to save this as a csv in your root folder (or you can edit filepath in second last row of the audio features function)
+ If you say yes to saving file, give a name to the csv file
                                     
Below, I ran the function to save data for Eminem's Relapse album!
I have also saved it into a variable to explore file variables for demonstration below.

In [16]:
playlist_df = audio_features()

Is it a Playlist or an Album? p/aa
Enter Album ID0iBl0xtWCSFHISzTmp6Wa3
Enter number of songs you want info for (between 1 and 100)27
Get only relevant data? y/ny
Write to csv? y/ny
Save by name?Relapse Refill


The Playlist or Album ID is at the end of the link.
https://open.spotify.com/playlist/37i9dQZF1DX5gQonLbZD9s
The Playlist ID here is 37i9dQZF1DX5gQonLbZD9s
    
You can get this link from your browser, or after hitting share from your Spotify app!

Here's a link to Eminem's profile, you'll find all his albums there:
https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR

### Merging all albums into one csv file

After saving all album's information into csv files, we'd need to concatenate them as one csv.

In [4]:
tsslp = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/The Slim Shady LP Expanded.csv')
tmmlp = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/The Marshall Mathers LP.csv')
tes = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/The Eminem Show.csv')
encore_deluxe = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Encore Deluxe.csv')
relapse_refill = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Relapse Refill.csv')
recovery = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Recovery.csv')
tmmlp2 = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/The Marshall Mathers LP2 Deluxe.csv')
revival = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Revival.csv')
Kamikaze = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Kamikaze.csv')
m2bmb = pd.read_csv('/Users/kaivalya/Spotify/Blog/Eminem/Music To Be Murdered By.csv')

In [5]:
frames_joined = [tsslp, tmmlp]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, tes]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, encore_deluxe]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, relapse_refill]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, recovery]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, tmmlp2]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, revival]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, Kamikaze]
eminem = pd.concat(frames_joined, ignore_index=True)
frames_joined = [eminem, m2bmb]
eminem = pd.concat(frames_joined, ignore_index=True)
eminem

,available_markets,disc_number,duration_ms_x,explicit,id,name,track_number,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,time_signature,album_name,album_released,album_peaked_on,days_to_peak,total_sales_US
0,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,33093,True,59z8uxWZVFpL2LfZ5C9AzY,Public Service Announcement,1,0.660,0.199,9,...,0.000000,0.1450,0.9210,147.535,1,The Slim Shady LP Expanded,2/23/99,3/13/99,18,"4,000,000"
1,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,268400,True,0Fv6CfegUbD5Egs0dj8CVB,My Name Is,2,0.845,0.682,9,...,0.000000,0.0931,0.8130,85.561,4,The Slim Shady LP Expanded,2/23/99,3/13/99,18,"4,000,000"
2,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,199333,True,2iW8D4Vdy8lrOEBkYEJOzd,Guilty Conscience,3,0.723,0.786,7,...,0.000000,0.3180,0.5160,90.237,4,The Slim Shady LP Expanded,2/23/99,3/13/99,18,"4,000,000"
3,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,226706,True,7iwK7RjGGJW9jdPj8ny754,Brain Damage,4,0.871,0.602,2,...,0.000000,0.0634,0.8470,94.989,4,The Slim Shady LP Expanded,2/23/99,3/13/99,18,"4,000,000"
4,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,15800,True,4QB28IqGl9W8sDcIK7eBIW,Paul - Skit,5,0.829,0.253,10,...,0.000094,0.7600,0.8090,97.478,3,The Slim Shady LP Expanded,2/23/99,3/13/99,18,"4,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,170226,True,1qwMkkRiD5jqLeUUjfgblh,Lock It Up (feat. Anderson .Paak),16,0.848,0.745,6,...,0.000000,0.0796,0.5160,137.024,4,Music To Be Murdered By,1/17/20,1/25/20,8,"325,000"
204,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,247733,True,74gykWHknHDrg4eMdcmdRq,Farewell,17,0.890,0.664,4,...,0.000000,0.2570,0.4670,93.007,4,Music To Be Murdered By,1/17/20,1/25/20,8,"325,000"
205,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,200946,True,1lO9CZo6eDrOy0S7khgryG,No Regrets (feat. Don Toliver),18,0.756,0.747,9,...,0.005360,0.5880,0.0687,134.935,4,Music To Be Murdered By,1/17/20,1/25/20,8,"325,000"
206,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",1,303000,True,3CJbxqRQ0JNCqboWDNUUeX,"I Will (feat. KXNG Crooked, Royce Da 5'9"" & Jo...",19,0.635,0.543,8,...,0.000678,0.2720,0.0360,98.743,4,Music To Be Murdered By,1/17/20,1/25/20,8,"325,000"


Now that all album's information is in one csv, we can export it and plug it into Tableau!

In [6]:
export_csv = eminem.to_csv (r'/Users/kaivalya/Spotify/Blog/Eminem/all.csv', index = None, header=True)

That was it! You can check out what I've done with this data on Tableau for inspiration.
+ https://public.tableau.com/profile/kaivalya.powale#!/
    
Or reach out to me directly and I'd love to talk!
+ https://www.linkedin.com/in/kaivalya-powale/
    
I would love to hear from you if you used this to create anything!